In [1]:
import os
import gensim
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import time
with open('../data/all_text.txt','r') as f:
    data = f.read().split(' ---------- ')

In [2]:
def read_corpus(docs, tokens_only=False):
    p = PorterStemmer()
    for i, line in enumerate(docs):
        line = remove_stopwords(line)
        line = p.stem_sentence(line)
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [3]:
t = time.time()
train_corpus = list(read_corpus(data))

In [4]:
model = Doc2Vec(vector_size=1024,
                epochs=50,
                workers=8,
                window=5,
                min_count=5,
                sample=1e-5,
                negative=5,
                dm=1)

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus,
            total_examples=model.corpus_count)

In [ ]:
print(time.time()-t)

In [ ]:
vector = model.infer_vector(data[5454].split(' '))

In [ ]:
vector